In [70]:
from keras.models import Sequential,Model
from keras.layers import Dense,Input,merge
from keras.layers import Reshape
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Deconvolution2D, Convolution2D,UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import SGD,Adam
from keras.datasets import mnist
import numpy as np
import math
from PIL import Image
from matplotlib.pyplot import imshow


%matplotlib inline



In [63]:
from keras import backend as K
from keras.engine.topology import Layer
from keras.engine import InputSpec
import numpy as np
import tensorflow as tf


class ReflectionPadding2D(Layer):
    def __init__(self,
                 padding=(1, 1),
                 dim_ordering='default',
                 **kwargs):
        super(ReflectionPadding2D, self).__init__(**kwargs)
        if dim_ordering == 'default':
            dim_ordering = K.image_dim_ordering()

        self.padding = padding
        if isinstance(padding, dict):
            if set(padding.keys()) <= {'top_pad', 'bottom_pad', 'left_pad', 'right_pad'}:
                self.top_pad = padding.get('top_pad', 0)
                self.bottom_pad = padding.get('bottom_pad', 0)
                self.left_pad = padding.get('left_pad', 0)
                self.right_pad = padding.get('right_pad', 0)
            else:
                raise ValueError('Unexpected key found in `padding` dictionary. '
                                 'Keys have to be in {"top_pad", "bottom_pad", '
                                 '"left_pad", "right_pad"}.'
                                 'Found: ' + str(padding.keys()))
        else:
            padding = tuple(padding)
            if len(padding) == 2:
                self.top_pad = padding[0]
                self.bottom_pad = padding[0]
                self.left_pad = padding[1]
                self.right_pad = padding[1]
            elif len(padding) == 4:
                self.top_pad = padding[0]
                self.bottom_pad = padding[1]
                self.left_pad = padding[2]
                self.right_pad = padding[3]
            else:
                raise TypeError('`padding` should be tuple of int '
                                'of length 2 or 4, or dict. '
                                'Found: ' + str(padding))

        if dim_ordering not in {'tf'}:
            raise ValueError('dim_ordering must be in {tf}.')
        self.dim_ordering = dim_ordering
        self.input_spec = [InputSpec(ndim=4)] 


    def call(self, x, mask=None):
        top_pad=self.top_pad
        bottom_pad=self.bottom_pad
        left_pad=self.left_pad
        right_pad=self.right_pad        
        
        paddings = [[0,0],[left_pad,right_pad],[top_pad,bottom_pad],[0,0]]

        
        return tf.pad(x,paddings, mode='REFLECT', name=None)

    def get_output_shape_for(self, input_shape):
        if self.dim_ordering == 'tf':
            rows = input_shape[1] + self.top_pad + self.bottom_pad if input_shape[1] is not None else None
            cols = input_shape[2] + self.left_pad + self.right_pad if input_shape[2] is not None else None

            return (input_shape[0],
                    rows,
                    cols,
                    input_shape[3])
        else:
            raise ValueError('Invalid dim_ordering:', self.dim_ordering)
            
    def get_config(self):
        config = {'padding': self.padding}
        base_config = super(ReflectionPadding2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))           
        

In [112]:
def conv_bn_relu(nb_filter, nb_row, nb_col,stride):   
    def conv_func(x):
        x = Convolution2D(nb_filter, nb_row, nb_col, subsample=stride,border_mode='same')(x)
        x = BatchNormalization(mode=1)(x)
        x = Activation('relu')(x)
        return x
    return conv_func    



#https://keunwoochoi.wordpress.com/2016/03/09/residual-networks-implementation-on-keras/
def res(nb_filter, nb_row, nb_col,stride=(1,1)):   
    def _shortcut(input):
        shortcut = Convolution2D(nb_filter=128, nb_row=3, nb_col=3, subsample=(1, 1), border_mode='valid')(input)
        shortcut = Convolution2D(nb_filter=128, nb_row=3, nb_col=3, subsample=(1, 1), border_mode='valid')(shortcut)
        return shortcut   
    
    def _res_func(x):
        a = Convolution2D(nb_filter, nb_row, nb_col, subsample=stride, border_mode='valid')(x)
        a = BatchNormalization(mode=1)(a)
        a = Activation('relu')(a)
        a = Convolution2D(nb_filter, nb_row, nb_col, subsample=stride, border_mode='valid')(a)
        y = BatchNormalization(mode=1)(a)
        
        shortcut = _shortcut(x)
        
        return merge([shortcut, y], mode='sum')
    
    return _res_func    


    model.add(UpSampling2D(size=(2, 2)))
    model.add(Convolution2D(64, 5, 5, border_mode='same'))
    model.add(Activation('tanh'))

    
def dconv_bn_relu(nb_filter, nb_row, nb_col,stride=(2,2)):   
    def dconv_bn_relu(x):
        x = UpSampling2D(size=stride)(x)
        x = Convolution2D(nb_filter,nb_row, nb_col, border_mode='same')(x)
        x = Activation('relu')(x)
        return x
    return dconv_bn_relu        


In [115]:
def image_transform_netl():
    x = Input(shape=(256,256,3))
    a = ReflectionPadding2D(padding=(40,40),input_shape=(256,256,3))(x)
    a = conv_bn_relu(32, 9, 9, stride=(1,1))(a)
    a = conv_bn_relu(64, 9, 9, stride=(2,2))(a)
    a = conv_bn_relu(128, 3, 3, stride=(2,2))(a)
    for i in range(5):
        a = res(128,3,3)(a)
    a = dconv_bn_relu(64,3,3)(a)
    a = dconv_bn_relu(32,3,3)(a)
    y = dconv_bn_relu(3,9,9,stride=(1,1))(a)
    return  Model(input=x, output=y)

# print image_transform_netl().summary() 


# Input 3 × 256 × 256
# Reflection Padding (40 × 40) 3 × 336 × 336
# 32 × 9 × 9 conv, stride 1 32 × 336 × 336
# 64 × 3 × 3 conv, stride 2 64 × 168 × 168
# 128 × 3 × 3 conv, stride 2 128 × 84 × 84
# Residual block, 128 filters 128 × 80 × 80
# Residual block, 128 filters 128 × 76 × 76
# Residual block, 128 filters 128 × 72 × 72
# Residual block, 128 filters 128 × 68 × 68
# Residual block, 128 filters 128 × 64 × 64
# 64 × 3 × 3 conv, stride 1/2 64 × 128 × 128
# 32 × 3 × 3 conv, stride 1/2 32 × 256 × 256
# 3 × 9 × 9 conv, stride 1 3 × 256 × 256

In [120]:
from keras.preprocessing.image import load_img, img_to_array
from scipy.misc import imsave
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg16
from keras import backend as K


def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg16.preprocess_input(img)
    return img

# util function to convert a tensor into a valid image


def deprocess_image(x):
    if K.image_dim_ordering() == 'th':
        x = x.reshape((3, img_nrows, img_ncols))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x



def gram_matrix(x):
    assert K.ndim(x) == 3
    if K.image_dim_ordering() == 'th':
        features = K.batch_flatten(x)
    else:
        features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# the "style loss" is designed to maintain
# the style of the reference image in the generated image.
# It is based on the gram matrices (which capture style) of
# feature maps from the style reference image
# and from the generated image


def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

# an auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image


def content_loss(base, combination):
    return K.sum(K.square(combination - base))

# the 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent


def total_variation_loss(x):
    assert K.ndim(x) == 4
    if K.image_dim_ordering() == 'th':
        a = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, 1:, :img_ncols - 1])
        b = K.square(x[:, :, :img_nrows - 1, :img_ncols - 1] - x[:, :, :img_nrows - 1, 1:])
    else:
        a = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
        b = K.square(x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))





In [122]:

# get tensor representations of our images
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))
    

# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

# build the VGG16 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg16.VGG16(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)


# combine these loss functions into a single scalar
loss = K.variable(0.)
layer_features = outputs_dict['block4_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss += content_weight * content_loss(base_image_features,
                                      combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss += (style_weight / len(feature_layers)) * sl
loss += total_variation_weight * total_variation_loss(combination_image)


def loss_net():
    x = Input(shape=(256,256,3))
    a = ReflectionPadding2D(padding=(40,40),input_shape=(256,256,3))(x)
    a = conv_bn_relu(32, 9, 9, stride=(1,1))(a)
    a = conv_bn_relu(64, 9, 9, stride=(2,2))(a)
    a = conv_bn_relu(128, 3, 3, stride=(2,2))(a)
    for i in range(5):
        a = res(128,3,3)(a)
    a = dconv_bn_relu(64,3,3)(a)
    a = dconv_bn_relu(32,3,3)(a)
    y = dconv_bn_relu(3,9,9,stride=(1,1))(a)
    return  Model(input=x, output=y)

NameError: name 'base_image_path' is not defined